# Module 6: Line/Lane Tracking

This is where we start to get to the main application for our Jetbot. We want to be able to detect the lane and find the trajectory for which our robot should follow. This will involve some of the lessons we have previously learned, and applying them in a separate context.

Below is our normal starter code for initializing all variables and packages. An image should appear to ensure it is working properly.

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
import ipywidgets as widgets
from scipy.signal import medfilt
from IPython.display import display, Image, clear_output
import time

robot = Robot()
# Initialize our camera
camera = Camera.instance()

# Save image data (modified to be numpy array)
image = np.array(camera.value)

jpeg_image = bgr8_to_jpeg(image)

# Display image within Jupyter Notebook
display(Image(data=jpeg_image))

## Method

Normally, for a regular autonomous vehicle, the car is programmed to identify the left and right lane markings, then develop the path and trajectory that the car should take. But this often involves multiple sensors and cameras that have both a wide field-of-view, as well as at various locations. But our Jetbot only has a singular camera, and as you may have seen in previous modules, the field-of-view is not particularly large. Thus, we are attempting a modified version, where we will detect the yellow middle markings, and have the Jetbot simply follow that dotted line.

## Implementation

To accomplish our task, we will undergo the following steps to our image.

1. HSV Color Masking

    One of the many ways to differentiate the yellow dotted line from the rest of the image, is to isolate the color yellow. To do this, we will use HSV color masking.

2. Sobel Edge Detection

    After we have isolated only the color yellow, we want to detect the edges, which highlight major changes in values between pixels. This also involves many other pre-processing techniques, including blurring and filtering based on area and direction.

3. Midpoint Calculation

    The Sobel Edge Detection will actually detect two vertical edges on each side of the yellow lane marker. But for our Jetbot, we want it to follow in the middle of these two points. This process is already completed, but it also involves filtering outliers and major deviations.

When you run the code below, a few widgets should appear below. These widgets will allow you to modify various values and parameters, which you can tune to find the best values for detecting the yellow lane markings.

In [ ]:
def adjust_gamma(image, gamma=1.2):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

def iqr_filter(x_list):
    x = np.array(x_list)
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return [val for val in x if lower <= val <= upper]

def get_yellow_lane_debug(image, apply_gamma=True, lower_yellow=(18, 80, 100), upper_yellow=(40, 255, 255)):
    h, w = image.shape[:2]

    if apply_gamma:
        invGamma = 1.0 / 1.2
        table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
        image = cv2.LUT(image, table)

    # === HSV yellow mask ===
    blur = cv2.bilateralFilter(image, 9, 75, 75)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    lower_yellow = (18, 80, 100)
    upper_yellow = (40, 255, 255)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Morphology to clean specks
    kernel = np.ones((5, 5), np.uint8)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=1)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Remove small blobs based on contour area
    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    clean_mask = np.zeros_like(mask_yellow)
    for cnt in contours:
        if cv2.contourArea(cnt) > 150:
            cv2.drawContours(clean_mask, [cnt], -1, 255, thickness=cv2.FILLED)
    mask_yellow = clean_mask

    # === Sobel X edges on yellow mask only ===
    yellow_mask_blur = cv2.GaussianBlur(mask_yellow, (5, 5), 0)
    sobelx = cv2.Sobel(yellow_mask_blur, cv2.CV_64F, 1, 0, ksize=3)
    sobelx = np.absolute(sobelx)
    sobelx = np.uint8(255 * sobelx / np.max(sobelx))
    _, edge_mask = cv2.threshold(sobelx, 50, 255, cv2.THRESH_BINARY)

    # === Get edge points ===
    midline_points = []
    left_points = []
    right_points = []

    for y in range(0, h, 4):
        x_candidates = [x for x in range(w) if edge_mask[y, x] > 0]
        if len(x_candidates) >= 3:
            x_candidates = iqr_filter(x_candidates)

            left_x = min(x_candidates)
            right_x = max(x_candidates)
            center_x = int(np.mean(x_candidates))

            midline_points.append((center_x, y))
            left_points.append((left_x, y))
            right_points.append((right_x, y))

    # === Midline smoothing ===
    if midline_points:
        x_vals = [x for x, y in midline_points]
        x_smooth = medfilt(x_vals, kernel_size=5)
        midline_points = list(zip(x_smooth, [y for _, y in midline_points]))

    # === Visualization ===
    overlay = image.copy()

    for (x, y) in midline_points:
        cv2.circle(overlay, (int(x), int(y)), 2, (0, 255, 0), -1)  # Green: midline
    for (x, y) in left_points:
        cv2.circle(overlay, (int(x), int(y)), 2, (255, 0, 0), -1)  # Blue: left edge
    for (x, y) in right_points:
        cv2.circle(overlay, (int(x), int(y)), 2, (0, 0, 255), -1)  # Red: right edge

    return overlay, mask_yellow, sobelx, edge_mask, midline_points

In [ ]:
import threading

# Create image widgets
overlay_widget = widgets.Image(format='jpeg', width=300, height=300)
yellow_mask_widget = widgets.Image(format='jpeg', width=300, height=300)
sobel_widget = widgets.Image(format='jpeg', width=300, height=300)
edge_mask_widget = widgets.Image(format='jpeg', width=300, height=300)

# Display them with labels
display(widgets.HBox([
    widgets.Label("Overlay with midline"),
    overlay_widget,
    widgets.Label("Yellow Mask"),
    yellow_mask_widget,
    widgets.Label("Sobel X"),
    sobel_widget,
    widgets.Label("Edge Mask"),
    edge_mask_widget
]))

# HSV sliders
h_lower = widgets.IntSlider(value=18, min=0, max=179, description='H Lower')
s_lower = widgets.IntSlider(value=80, min=0, max=255, description='S Lower')
v_lower = widgets.IntSlider(value=100, min=0, max=255, description='V Lower')

h_upper = widgets.IntSlider(value=40, min=0, max=179, description='H Upper')
s_upper = widgets.IntSlider(value=255, min=0, max=255, description='S Upper')
v_upper = widgets.IntSlider(value=255, min=0, max=255, description='V Upper')

hsv_controls = widgets.VBox([
    widgets.Label("Adjust HSV Thresholds"),
    h_lower, s_lower, v_lower,
    h_upper, s_upper, v_upper
])

display(hsv_controls)


# === Threaded Image Update ===
def update_images():
    while True:
        img = camera.value
        overlay, yellow_mask, sobelx, edge_mask, _ = get_yellow_lane_debug(img)

        overlay_widget.value = bgr8_to_jpeg(overlay)
        yellow_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(yellow_mask, cv2.COLOR_GRAY2BGR))
        sobel_widget.value = bgr8_to_jpeg(cv2.cvtColor(sobelx, cv2.COLOR_GRAY2BGR))
        edge_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(edge_mask, cv2.COLOR_GRAY2BGR))

        time.sleep(0.1)  # Adjust refresh rate as needed

threading.Thread(target=update_images, daemon=True).start()


threading.Thread(target=update_images, daemon=True).start()

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
import ipywidgets as widgets
from scipy.signal import medfilt
from IPython.display import display
import threading
import time

robot = Robot()
camera = Camera.instance()

# === HSV Control Sliders ===
h_min = widgets.IntSlider(value=18, min=0, max=179, description='H Min')
s_min = widgets.IntSlider(value=80, min=0, max=255, description='S Min')
v_min = widgets.IntSlider(value=100, min=0, max=255, description='V Min')
h_max = widgets.IntSlider(value=40, min=0, max=179, description='H Max')
s_max = widgets.IntSlider(value=255, min=0, max=255, description='S Max')
v_max = widgets.IntSlider(value=255, min=0, max=255, description='V Max')

display(widgets.VBox([
    widgets.Label("Adjust HSV range for yellow detection:"),
    h_min, h_max,
    s_min, s_max,
    v_min, v_max
]))

# === Image display widgets ===
overlay_widget = widgets.Image(format='jpeg', width=300, height=300)
yellow_mask_widget = widgets.Image(format='jpeg', width=300, height=300)
sobel_widget = widgets.Image(format='jpeg', width=300, height=300)
edge_mask_widget = widgets.Image(format='jpeg', width=300, height=300)

display(widgets.HBox([
    widgets.VBox([widgets.Label("Overlay with midline"), overlay_widget]),
    widgets.VBox([widgets.Label("Yellow Mask"), yellow_mask_widget]),
    widgets.VBox([widgets.Label("Sobel X"), sobel_widget]),
    widgets.VBox([widgets.Label("Edge Mask"), edge_mask_widget])
]))

def adjust_gamma(image, gamma=1.2):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

def iqr_filter(x_list):
    x = np.array(x_list)
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return [val for val in x if lower <= val <= upper]

def get_yellow_lane_debug(image, apply_gamma=True):
    h, w = image.shape[:2]

    if apply_gamma:
        image = adjust_gamma(image, gamma=1.2)

    blur = cv2.bilateralFilter(image, 9, 75, 75)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    # Read HSV range from sliders
    lower_yellow = (h_min.value, s_min.value, v_min.value)
    upper_yellow = (h_max.value, s_max.value, v_max.value)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Morphology to clean specks
    kernel = np.ones((5, 5), np.uint8)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=1)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Remove small blobs
    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    clean_mask = np.zeros_like(mask_yellow)
    for cnt in contours:
        if cv2.contourArea(cnt) > 150:
            cv2.drawContours(clean_mask, [cnt], -1, 255, thickness=cv2.FILLED)
    mask_yellow = clean_mask

    # Edge detection
    yellow_mask_blur = cv2.GaussianBlur(mask_yellow, (5, 5), 0)
    sobelx = cv2.Sobel(yellow_mask_blur, cv2.CV_64F, 1, 0, ksize=3)
    sobelx = np.absolute(sobelx)
    if np.max(sobelx) > 0:
        sobelx = np.uint8(255 * sobelx / np.max(sobelx))
    else:
        sobelx = np.zeros_like(sobelx, dtype=np.uint8)
    _, edge_mask = cv2.threshold(sobelx, 50, 255, cv2.THRESH_BINARY)

    midline_points = []
    left_points = []
    right_points = []

    for y in range(0, h, 4):
        x_candidates = [x for x in range(w) if edge_mask[y, x] > 0]
        if len(x_candidates) >= 3:
            x_candidates = iqr_filter(x_candidates)
            left_x = min(x_candidates)
            right_x = max(x_candidates)
            center_x = int(np.mean(x_candidates))

            midline_points.append((center_x, y))
            left_points.append((left_x, y))
            right_points.append((right_x, y))

    if midline_points:
        x_vals = [x for x, _ in midline_points]
        x_smooth = medfilt(x_vals, kernel_size=5)
        midline_points = list(zip(x_smooth, [y for _, y in midline_points]))

    overlay = image.copy()
    for (x, y) in midline_points:
        cv2.circle(overlay, (int(x), int(y)), 2, (0, 255, 0), -1)
    for (x, y) in left_points:
        cv2.circle(overlay, (int(x), int(y)), 2, (255, 0, 0), -1)
    for (x, y) in right_points:
        cv2.circle(overlay, (int(x), int(y)), 2, (0, 0, 255), -1)

    return overlay, mask_yellow, sobelx, edge_mask

# === Periodic UI-safe update ===
from IPython.display import clear_output
from threading import Event

stop_event = Event()

def update_loop():
    while not stop_event.is_set():
        frame = camera.value
        overlay, yellow_mask, sobelx, edge_mask = get_yellow_lane_debug(frame)

        # Ensure UI-safe conversions
        overlay_widget.value = bgr8_to_jpeg(overlay)
        yellow_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(yellow_mask, cv2.COLOR_GRAY2BGR))
        sobel_uint8 = np.uint8(sobelx)
        sobel_widget.value = bgr8_to_jpeg(cv2.cvtColor(sobel_uint8, cv2.COLOR_GRAY2BGR))
        edge_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(edge_mask, cv2.COLOR_GRAY2BGR))

        time.sleep(0.1)

thread = threading.Thread(target=update_loop, daemon=True)
thread.start()

## Bonus Challenge

If you want an extra challenge, see if you can modify the code above to detect the left and right lane and find the midpoint trajectory from those lanes!

<div class="alert alert-block alert-warning">
<b>INCLUDE CODE TO CALCULATE MIDPOINT FOR BONUS CHALLENGE</b>
</div>